In [142]:
import torch, torchvision
from torch import nn
from torch import optim
from torchvision.transforms import ToTensor, transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt

import copy

from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
import random 

from PIL import Image
import os.path
from os import path
import glob
import numpy
import random
from torch.utils.data import Dataset, DataLoader

BATCH_SIZE = 4

In [144]:
class CatfishDataset(Dataset):
    def __init__(self, images, values):
        self.images = images
        self.labels = values
    def __len__(self):
        return len(self.images)
    def __getitem__(self, index):
        image = self.images[index]
        label = self.labels[index]
        return image, label
    

images = torch.load("C:/Users/nikce/anaconda3/Project/images/inputVars.pt")
values = torch.load("C:/Users/nikce/anaconda3/Project/images/outputVars.pt")    
images = images.reshape(1043, 1, 200, 407)
values = values.reshape(1043, 7)

train_images = images[0:399]
test_images = images[400:1042]

train_values = values[0:399]
test_values = values[400:1042]

train_dataset = CatfishDataset(train_images, train_values)
test_dataset = CatfishDataset(test_images, test_values)

train_dl = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True)
test_dl = DataLoader(test_dataset, batch_size = BATCH_SIZE, shuffle = True)

    


In [145]:
def create_lenet():
    model = nn.Sequential(
        
    nn.Conv2d(1, 6, 5, padding = 2),
    nn.ReLU(),
    nn.AvgPool2d(2, stride = 2),
    
    nn.Conv2d(6, 16, 5, padding = 0),
    nn.ReLU(),
    nn.AvgPool2d(2, stride = 2), 
    
    nn.Flatten(),
    nn.Linear(76032, 120),
    nn.ReLU(),
    nn.Linear(120, 84),
    nn.ReLU(),
    nn.Linear(84, 7)
)
    return model

In [146]:
def validate(model, data):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(data):
        x = model(images)
        value, pred = torch.max(x, 1)
        total += x.size(0)
        useless, labels_index = torch.max(labels,1)
        correct += torch.sum(pred == labels_index)
    
    return correct*100./total

In [147]:
def train(lr, epochs, device = "cpu"):
    accuracies = []
    cnn = create_lenet().to(device)
    cec = nn.CrossEntropyLoss()
    optimizer = optim.Adam(cnn.parameters(), lr = lr)
    max_accuracy = 0
    
    for epoch in range(epochs):
        for i, (images, labels) in enumerate(train_dl):
            optimizer.zero_grad()
            pred = cnn(images)
            loss = cec(pred, labels)
            loss.backward()
            optimizer.step()
        accuracy = float(validate(cnn, test_dl))
        accuracies.append(accuracy)
        if accuracy > max_accuracy:
            best_model = copy.deepcopy(cnn)
            max_accuracy = accuracy 
            print("Saving Best Model With Accuracy: ", accuracy)
        print("Epoch: ", epoch+1, "Accuracy: ", accuracy, "%")
    #plt.plot(accuracies)
    return best_model, accuracies, lr

In [109]:
lenet = train(0.001, 100) # used image 1-200 to train and 800-1043 to test
# model is 68% accurate on train and test images, but fails to generalize to 
# images 201-799

Epoch:  1 Accuracy:  0.0 %
Saving Best Model With Accuracy:  4.13223123550415
Epoch:  2 Accuracy:  4.13223123550415 %
Saving Best Model With Accuracy:  17.76859474182129
Epoch:  3 Accuracy:  17.76859474182129 %
Saving Best Model With Accuracy:  59.50413131713867
Epoch:  4 Accuracy:  59.50413131713867 %
Saving Best Model With Accuracy:  68.18181610107422
Epoch:  5 Accuracy:  68.18181610107422 %
Epoch:  6 Accuracy:  66.1156997680664 %
Epoch:  7 Accuracy:  40.49586868286133 %
Epoch:  8 Accuracy:  24.380165100097656 %
Epoch:  9 Accuracy:  16.11570167541504 %
Epoch:  10 Accuracy:  40.082645416259766 %
Epoch:  11 Accuracy:  40.49586868286133 %
Epoch:  12 Accuracy:  35.95041275024414 %
Epoch:  13 Accuracy:  46.6942138671875 %
Epoch:  14 Accuracy:  44.214874267578125 %
Epoch:  15 Accuracy:  47.10743713378906 %
Epoch:  16 Accuracy:  46.6942138671875 %
Epoch:  17 Accuracy:  47.10743713378906 %
Epoch:  18 Accuracy:  47.10743713378906 %
Epoch:  19 Accuracy:  47.520660400390625 %
Epoch:  20 Accurac

In [148]:
lenet2 = train(0.001, 10)

Saving Best Model With Accuracy:  21.183799743652344
Epoch:  1 Accuracy:  21.183799743652344 %
Epoch:  2 Accuracy:  21.183799743652344 %
Epoch:  3 Accuracy:  21.183799743652344 %
Epoch:  4 Accuracy:  21.183799743652344 %
Epoch:  5 Accuracy:  21.183799743652344 %
Epoch:  6 Accuracy:  21.183799743652344 %
Epoch:  7 Accuracy:  21.183799743652344 %
Epoch:  8 Accuracy:  21.183799743652344 %
Epoch:  9 Accuracy:  21.183799743652344 %
Epoch:  10 Accuracy:  21.183799743652344 %


In [155]:
x = images[1030]
x = x.reshape(1, 1, 200, 407)
print(lenet[0](x))

tensor([[-3.5123e+00, -5.1998e-01,  6.2475e-04,  6.5293e-01,  7.7898e-01,
         -2.8068e+00, -3.5687e-01]], grad_fn=<AddmmBackward0>)


In [156]:
def predict_dl(model, data):
    y_pred = []
    y_true = []
    for i, (images, labels) in enumerate(data):
        x = model(images)
        value, pred = torch.max(x, 1)
        pred = pred.data.cpu()
        y_pred.extend(list(pred.numpy()))
        y_true.extend(list(labels.numpy()))
    return np.array(y_pred), np.array(y_true)

In [157]:
y_pred, y_true = predict_dl(lenet[0], test_dl)

In [158]:
pd.DataFrame(confusion_matrix(y_true, y_pred, labels = np.arange(1, 7)))

ValueError: Classification metrics can't handle a mix of multilabel-indicator and multiclass targets